In [1]:
import os
# import tensorflow.compat.v2 as tf
import tensorflow as tf
import json
from tensorflow.keras import datasets, layers, models, metrics
from minio import Minio
from minio.error import ResponseError

In [2]:
minioClient = Minio('minio-service.kubeflow:9000',
              access_key='minio', secret_key='minio123', secure=False)
try:
    data = minioClient.get_object('dataset', 'mnist/mnist.npz')
    with open('/tmp/mnist.npz', 'wb') as file_data:
        for d in data.stream(32*1024):
            file_data.write(d)
except ResponseError as err:
    print(err)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='/tmp/mnist.npz')
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
dropout_rate = 0.3
learning_rate = 0.1

In [4]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

sgd = tf.keras.optimizers.SGD(lr=learning_rate,
                                  decay=1e-6,
                                  momentum=0.9,
                                  nesterov=True)
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10)
score = model.evaluate(x_test,  y_test, verbose=2)
loss = score[0]
accuracy = score[1]

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4349 - accuracy: 0.8733
Epoch 2/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3451 - accuracy: 0.9050
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3086 - accuracy: 0.9168
Epoch 4/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2908 - accuracy: 0.9241
Epoch 5/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2796 - accuracy: 0.9287
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2612 - accuracy: 0.9332
Epoch 7/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2522 - accuracy: 0.9348
Epoch 8/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2588 - accuracy: 0.9346
Epoch 9/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2470 - accuracy: 0.9377
Epoch 10/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.244

In [8]:
model.save('haha.h5')

os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minio',
    'AWS_SECRET_ACCESS_KEY': 'minio123',
    'S3_USE_HTTPS'         : '0',	# Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0' 	# If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})    
model.save("s3://model/mnist-model")

In [15]:
from tensorflow.keras.models import save_model

os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minio',
    'AWS_SECRET_ACCESS_KEY': 'minio123',
    'S3_USE_HTTPS'         : '0',	# Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0' 	# If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})    

save_model(model, "s3://model/mnist-model/1")

DataLossError: truncated block read [Op:MergeV2Checkpoints]

In [6]:
print(loss)
print(accuracy)

0.20902802050113678
0.9574999809265137
